In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib qt

In [ ]:
import os
import sys
from pathlib import Path

src_dir = str(Path(os.getcwd()).parent / "src")
sys.path.insert(1, src_dir)
#os.environ["PYTHONPATH"] = src_dir

In [ ]:
import data_loader
import calculate_concentr_diffs
from concentr_diffs_pathlength import *
from plotting import *
from mbll_functions import *
from utils import *
from mcs_function import *
import config
import pickle
from ray import tune
import matplotlib.pyplot as plt

load the gray-matter, and blood-vessel pathlengths and scattering counts

In [ ]:
gray_matter_mcs_pathlength_data = np.loadtxt(config.pl_dir / "gray_matter_mcs.txt")
blood_vessel_mcs_pathlength_data = np.loadtxt(config.pl_dir / "blood_vessel_pure_mcs.txt")

create MCS-Attenuation Object

In [ ]:
A_mcs = SimulationAttenuation(config.mcs_func_path)

load the m-parameters for the Jacques models

In [ ]:
with open(config.m_params_path, "rb") as f:
    m_params, _, _, _ = pickle.load(f)

# convert to more convenient dictionary
refraction_idx_dict = {"gray matter": 1.36, "tumor": 1.36, "artery": 1.4, "vein": 1.4, "general": 1.38}
jacques_m_params = {key : m_params[refraction_idx] for (key, refraction_idx) in refraction_idx_dict.items()}

# Forward Test

- Input tissue concentrations, and reconstruct spectra with every model
- compute squared average (over wavelength range) attenuation error per tissue and model

In [ ]:
with open(config.simulated_dataset_dir / "simulation_data.pickle", "rb") as f:
    wavelengths, simulation_data = pickle.load(f)

Interpolate path and scatter lengths for MBLL + MCS

In [ ]:
lengths = tuple()
for mcs_data in (gray_matter_mcs_pathlength_data, blood_vessel_mcs_pathlength_data):
    pathlength = np.interp(wavelengths, mcs_data[:, 0], mcs_data[:, 4])
    # notice the minus: mean scatterlength = mean pathlength - ((mean scattercount)/mu_s)
    scatterlength = -np.interp(wavelengths, mcs_data[:, 0], mcs_data[:, 5])
    scatterlength /= mcs_data[:, 2]
    scatterlength += pathlength
    lengths += (pathlength, scatterlength)

gray_matter_pathlength, gray_matter_scatterlength, blood_vessel_pathlength, blood_vessel_scatterlength = lengths

To test the MBLL with wavelength-independent PL, the average path- and scatter-lengths are computed

In [ ]:
gray_matter_pathlength_average = np.repeat(np.average(gray_matter_pathlength), len(wavelengths))
gray_matter_average_scatterlength = np.repeat(np.average(gray_matter_scatterlength), len(wavelengths))
blood_vessel_pathlength_average = np.repeat(np.average(blood_vessel_pathlength), len(wavelengths))
blood_vessel_average_scatterlength = np.repeat(np.average(blood_vessel_scatterlength), len(wavelengths))

all models and their tissue-specific inputs

In [ ]:
dict(p1=1, p2=2)

In [ ]:
tissue_types = ["gray matter", "artery", "vein", "tumor"]
forward_models = {
        mbll_new : 
        {
            "gray matter": dict(
                baseline_attenuation=simulation_data["gray matter"][2][:, 0],
                baseline_c=simulation_data["gray_matter"][0][:, 0],
                baseline_a=simulation_data["gray_matter"][1][:, 0],
                baseline_b=simulation_data["gray_matter"][1][:, 1],
                pathlength=gray_matter_pathlength,
                scatterlength=gray_matter_pathlength
            ),
            "artery": dict(
                baseline_attenuation=simulation_data["blood vessel average"][2][:, 0],
                baseline_c=simulation_data["blood vessel average"][0][:, 0],
                baseline_a=simulation_data["blood vessel average"][1][:, 0],
                baseline_b=simulation_data["blood vessel average"][1][:, 1],
                pathlength=blood_vessel_pathlength,
                scatterlength=blood_vessel_scatterlength
            ),
            "vein": dict(
                baseline_attenuation=simulation_data["blood vessel average"][2][:, 0],
                baseline_c=simulation_data["blood vessel average"][0][:, 0],
                baseline_a=simulation_data["blood vessel average"][1][:, 0],
                baseline_b=simulation_data["blood vessel average"][1][:, 1],
                pathlength=blood_vessel_pathlength,
                scatterlength=blood_vessel_scatterlength
            ),
            "tumor": dict(
                baseline_attenuation=simulation_data["gray matter"][2][:, 0],
                baseline_c=simulation_data["gray_matter"][0][:, 0],
                baseline_a=simulation_data["gray_matter"][1][:, 0],
                baseline_b=simulation_data["gray_matter"][1][:, 1],
                pathlength=gray_matter_pathlength,
                scatterlength=gray_matter_scatterlength
            )
        },
        # mbll but with constant path- and scatterlengths
        mbll_new :
        {
            "gray matter": dict(
                baseline_attenuation=simulation_data["gray matter"][2][:, 0],
                baseline_c=simulation_data["gray_matter"][0][:, 0],
                baseline_a=simulation_data["gray_matter"][1][:, 0],
                baseline_b=simulation_data["gray_matter"][1][:, 1],
                pathlength=gray_matter_pathlength_average,
                scatterlength=gray_matter_pathlength_average
            ),
            "artery": dict(
                baseline_attenuation=simulation_data["blood vessel average"][2][:, 0],
                baseline_c=simulation_data["blood vessel average"][0][:, 0],
                baseline_a=simulation_data["blood vessel average"][1][:, 0],
                baseline_b=simulation_data["blood vessel average"][1][:, 1],
                pathlength=blood_vessel_pathlength_average,
                scatterlength=blood_vessel_average_scatterlength
            ),
            "vein": dict(
                baseline_attenuation=simulation_data["blood vessel average"][2][:, 0],
                baseline_c=simulation_data["blood vessel average"][0][:, 0],
                baseline_a=simulation_data["blood vessel average"][1][:, 0],
                baseline_b=simulation_data["blood vessel average"][1][:, 1],
                pathlength=blood_vessel_pathlength_average,
                scatterlength=blood_vessel_average_scatterlength
            ),
            "tumor": dict(
                baseline_attenuation=simulation_data["gray matter"][2][:, 0],
                baseline_c=simulation_data["gray_matter"][0][:, 0],
                baseline_a=simulation_data["gray_matter"][1][:, 0],
                baseline_b=simulation_data["gray_matter"][1][:, 1],
                pathlength=gray_matter_pathlength_average,
                scatterlength=gray_matter_average_scatterlength
            )
        },
        # Jacques with tissue-specific m-parameters
        A_jacques_new :
        {tissue_type : {f"m{i}" : jacques_m_params[tissue_type][i-1] for i in range(1, 4)} for tissue_type in tissue_types},
        # Jacques with general m-parameters
        A_jacques_new :
        {tissue_type : {f"m{i}" : jacques_m_params["general"][i-1] for i in range(1, 4)} for tissue_type in tissue_types},          
        # White Monte Carlo, mus interpolated
        A_mcs.A_concentrations :
        {tissue_type: dict() for tissue_type in tissue_types},
        # diffusion equation by carp
        A_carp_concentrations :
        {tissue_type : {("g", "n")[i-2] : data_loader.DataLoader.tissue_parameters[tissue_type][i] for i in range(2, 4)} for tissue_type in tissue_types},
        # diffusion equation by patterson
        A_patterson_concentrations:
        {tissue_type : {"n" : data_loader.DataLoader.tissue_parameters[tissue_type][-1]} for tissue_type in tissue_types}
}


In [ ]:
#(model, tissue-type, num_spectra, wavelengths)
reconstructed_spectra = np.empty(len(forward_models), len(simulation_data.keys()), simulation_data["gray matter"][-1].shape[1], len(wavelengths))

In [ ]:
mu_a_matrix = data_loader.DataLoader.absorption_coefs(
    wavelengths,
    use_diff_oxycco=False,
    use_water_and_fat=True
)

In [ ]:
for model_idx, model_func in enumerate(forward_models.keys()):
    for tissue_idx, tissue_type in enumerate(tissue_types):
        for spectrum_idx, (concentrations, scattering_params) in enumerate(zip(*simulation_data[tissue_type][0][:2])):
            A_reconstructed = model_func(
                wavelengths,
                mu_a_matrix,
                concentrations,
                *scattering_params,
                **forward_models[model_func][tissue_type]
            )
            reconstructed_spectra[model_idx, tissue_idx, spectrum_idx, :] = A_reconstructed

Plot reconstructed spectra

In [ ]:
# convert to (model, tissue-type, wavelengths, spectrum-idx)
plot_data = reconstructed_spectra.transpose(0, 1, 3, 2)

for tissue_idx, tissue_type in enumerate(tissue_types):
    simulation_data[tissue_type][2]
    plot_spectra_slider(
        np.concatenate(
            (
                simulation_data[tissue_type][2],
                plot_data[:, tissue_idx, :, :]
            ),
            axis=0
        ),
        wavelengths,
        ["MCS GT", "MBLL", "MBLL const. PL", "Jacques", "Jacques general", "MCS interp.", "Carp", "Patterson"],
        title=tissue_type
    )

Store the results